In [1]:
import zipfile
import os
import pandas as pd

In [ ]:
# ---- CONFIG ----
# Path to the outer .zip file
data_zip_path = "multi_step_attacks_dataset.zip"

# Extraction folder
extract_dir = "extracted_dataset"
inner_extract_dir = os.path.join(extract_dir, "inner")
parquet_filename = "flattened_dataset.parquet"

# ---------------- Extract outer ZIP ----------------
with zipfile.ZipFile(data_zip_path, 'r') as outer_zip:
    outer_zip.extractall(extract_dir)

In [3]:
# Find inner zip file
inner_zip_path = None
for root, dirs, files in os.walk(extract_dir):
    for f in files:
        if f.endswith(".zip"):
            inner_zip_path = os.path.join(root, f)
            break


if inner_zip_path:
    os.makedirs(inner_extract_dir, exist_ok=True)
    with zipfile.ZipFile(inner_zip_path, 'r') as inner_zip:
        inner_zip.extractall(inner_extract_dir)

    # Delete all JSON files that end with 'aminer.json'
    for root, dirs, files in os.walk(inner_extract_dir):
        for f in files:
            if f.endswith("aminer.json"):
                file_path = os.path.join(root, f)
                try:
                    os.remove(file_path)
                    print(f"Deleted {file_path}")
                except Exception as e:
                    print(f"Could not delete {file_path}: {e}")
else:
    raise FileNotFoundError("No inner zip file found inside dataset.")


Deleted extracted_dataset\inner\fox_aminer.json
Deleted extracted_dataset\inner\harrison_aminer.json
Deleted extracted_dataset\inner\russellmitchell_aminer.json
Deleted extracted_dataset\inner\santos_aminer.json
Deleted extracted_dataset\inner\shaw_aminer.json
Deleted extracted_dataset\inner\wardbeck_aminer.json
Deleted extracted_dataset\inner\wheeler_aminer.json
Deleted extracted_dataset\inner\wilson_aminer.json


In [4]:
# ---------------- Load labels.csv ----------------
labels_path = os.path.join(extract_dir, "labels.csv")
if os.path.exists(labels_path):
    labels_df = pd.read_csv(labels_path, sep=",")
    print("\n--- Labels Dataset Info ---")
    print(labels_df.head())
    print("\nShape:", labels_df.shape)
    print("\nFields:", list(labels_df.columns))
else:
    print("labels.csv not found inside inner zip")


--- Labels Dataset Info ---
          scenario         attack         start           end
0  russellmitchell  network_scans  1.642993e+09  1.642997e+09
1  russellmitchell  service_scans  1.642997e+09  1.642997e+09
2  russellmitchell           dirb  1.642997e+09  1.642997e+09
3  russellmitchell         wpscan  1.642997e+09  1.642997e+09
4  russellmitchell       webshell  1.642997e+09  1.642997e+09

Shape: (79, 4)

Fields: ['scenario', 'attack', 'start', 'end']


In [ ]:
# ---------------- Load JSON files ----------------

all_dfs = []

for root, dirs, files in os.walk(inner_extract_dir):
    for f in files:
        if f.endswith(".json"):
            json_path = os.path.join(root, f)
            try:
                df = pd.read_json(json_path, lines=True)  # NDJSON support
                df['filename'] = f.split("_")[0]  # Add source filename column
                all_dfs.append(df)
                print(f"Loaded {f} with {len(df)} rows")
            except Exception as e:
                print(f"Could not read {f}: {e}")

# Combine into one big DataFrame
data = pd.concat(all_dfs, ignore_index=True)
print("\n--- Combined Dataset ---")
print(data.info())
print(data.columns)
    

Loaded fox_wazuh.json with 462523 rows
Loaded harrison_wazuh.json with 583754 rows
Loaded russellmitchell_wazuh.json with 41488 rows
Loaded santos_wazuh.json with 126513 rows
Loaded shaw_wazuh.json with 68539 rows
Loaded wardbeck_wazuh.json with 88204 rows
Loaded wheeler_wazuh.json with 603939 rows
Loaded wilson_wazuh.json with 625303 rows

--- Combined Dataset ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2600263 entries, 0 to 2600262
Data columns (total 14 columns):
 #   Column           Dtype  
---  ------           -----  
 0   predecoder       object 
 1   agent            object 
 2   manager          object 
 3   rule             object 
 4   decoder          object 
 5   full_log         object 
 6   input            object 
 7   @timestamp       object 
 8   location         object 
 9   id               float64
 10  data             object 
 11  GeoLocation      object 
 12  previous_output  object 
 13  filename         object 
dtypes: float64(1), object(13)
memory u

In [7]:
# ---------------- Flatten nested fields ----------------

unflattened_cols = ['predecoder', 'agent', 'manager', 'data', 'rule', 'decoder', 'input', 'GeoLocation']
flattened_dfs = []

# Function to flatten the 'mitre' dictionary and prefix keys
def flatten_mitre(m):
    # Ensure m is a dict before iterating. Return empty dict for non-dict (like NaN)
    if isinstance(m, dict):
        return {f"rule_mitre_{k}": v for k, v in m.items()}
    return {}

for col in unflattened_cols:
    if col in data.columns:
        # Convert to list of dictionaries for json_normalize, handling NaNs
        data_to_normalize = data[col].dropna().tolist()
        
        # Create a mapping from original index to the normalized data index
        original_indices = data[col].dropna().index
        
        # Flatten the data
        df_col = pd.json_normalize(data_to_normalize, sep='_')
        
        # Re-assign the original index to df_col so it aligns with data
        df_col.index = original_indices
        
        print(f"Columns in {col}:", df_col.columns.tolist())
        
        if col == 'rule' and 'mitre' in df_col.columns:
            # Apply flattening function to 'mitre' column and expand to new columns
            mitre_expanded = df_col['mitre'].apply(flatten_mitre).apply(pd.Series)
            
            # Drop the original 'mitre' column
            df_col = df_col.drop(columns=['mitre'])
            
            # Concatenate the expanded mitre columns. The indices will align.
            df_col = pd.concat([df_col, mitre_expanded], axis=1)
        
        # Handle cases where the original series had NaNs and the resulting df_col is shorter
        cols_to_rename = [c for c in df_col.columns if not c.startswith('rule_mitre_')]
        df_col.rename(columns={c: f"{col}_{c}" for c in cols_to_rename}, inplace=True)
        
        flattened_dfs.append(df_col)

# The key is that all DataFrames now share the same indices (or a subset thereof),
# so pd.concat(axis=1) will align them correctly.
df_flat = pd.concat([data.drop(columns=[c for c in unflattened_cols if c in data.columns])] + flattened_dfs, axis=1)

df_flat.head()

Columns in predecoder: ['hostname', 'program_name', 'timestamp']
Columns in agent: ['ip', 'name', 'id']
Columns in manager: ['name']
Columns in data: ['srcip', 'dstip', 'id', 'tx_id', 'app_proto', 'in_iface', 'src_ip', 'src_port', 'event_type', 'flow_id', 'dest_ip', 'proto', 'dest_port', 'timestamp', 'alert_severity', 'alert_signature_id', 'alert_rev', 'alert_metadata_updated_at', 'alert_metadata_former_category', 'alert_metadata_created_at', 'alert_gid', 'alert_signature', 'alert_action', 'alert_category', 'http_hostname', 'http_protocol', 'http_http_method', 'http_length', 'http_url', 'http_http_user_agent', 'http_status', 'flow_pkts_toserver', 'flow_start', 'flow_bytes_toclient', 'flow_bytes_toserver', 'flow_pkts_toclient', 'audit_uid', 'audit_auid', 'audit_gid', 'audit_exe', 'audit_euid', 'audit_session', 'audit_pid', 'audit_id', 'audit_type', 'audit_command', 'metadata_flowints_tls_anomaly_count', 'tls_version', 'tls_ja3_string', 'tls_ja3_hash', 'tls_sni', 'dstuser', 'srcport', 'u

,full_log,@timestamp,location,id,previous_output,filename,predecoder_hostname,predecoder_program_name,predecoder_timestamp,agent_ip,...,rule_mitre_tactic,decoder_name,decoder_parent,decoder_ftscomment,input_type,GeoLocation_city_name,GeoLocation_country_name,GeoLocation_region_name,GeoLocation_location_lon,GeoLocation_location_lat
0,Jan 15 02:32:32 mail freshclam[29266]: Sat Jan...,2022-01-15T02:32:32.000000Z,/var/log/syslog,1.686147e+09,NaN,fox,mail,freshclam,Jan 15 02:32:32,172.17.131.81,...,NaN,freshclam,NaN,NaN,log,NaN,NaN,NaN,NaN,NaN
1,Jan 15 02:32:32 taylorcruz-mail freshclam[2851...,2022-01-15T02:32:32.000000Z,/var/log/syslog,1.686147e+09,NaN,fox,taylorcruz-mail,freshclam,Jan 15 02:32:32,192.168.128.170,...,NaN,freshclam,NaN,NaN,log,NaN,NaN,NaN,NaN,NaN
2,Jan 15 02:32:37 mail freshclam[29266]: Sat Jan...,2022-01-15T02:32:37.000000Z,/var/log/syslog,1.686147e+09,NaN,fox,mail,freshclam,Jan 15 02:32:37,172.17.131.81,...,NaN,freshclam,NaN,NaN,log,NaN,NaN,NaN,NaN,NaN
3,Jan 15 02:32:42 mail freshclam[29266]: Sat Jan...,2022-01-15T02:32:42.000000Z,/var/log/syslog,1.686147e+09,NaN,fox,mail,freshclam,Jan 15 02:32:42,172.17.131.81,...,NaN,freshclam,NaN,NaN,log,NaN,NaN,NaN,NaN,NaN
4,Jan 15 02:32:47 mail freshclam[29266]: Sat Jan...,2022-01-15T02:32:47.000000Z,/var/log/syslog,1.686147e+09,NaN,fox,mail,freshclam,Jan 15 02:32:47,172.17.131.81,...,NaN,freshclam,NaN,NaN,log,NaN,NaN,NaN,NaN,NaN


In [8]:
# Save flattened data into a parquet file
df_flat.to_parquet(path=os.path.join(extract_dir, parquet_filename))